In [1]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [2]:
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips.main.size['size_z'] = '-500um'
design.chips.main.size['sample_holder_bottom'] = '500um'
design.chips.main.size['sample_holder_top'] = '2000um'



# Set hfss as renderer for design
#hfss = design.renderers.hfss
# Allow overwriting
gui = MetalGUI(design)
design.overwrite_enabled = True

#design.delete_all_components()

In [3]:
#Add hanger of capacitively coupled transmission lines
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.9mm',
                     pos_y='0mm',
                     coupling_length='200um',
                     coupling_space='1um',
                     rotation = '90',
                     open_termination=False))

gui.rebuild()
gui.autoscale()

In [4]:
otg1 = OpenToGround(design, 'open_to_ground', options=dict(pos_x='0.7mm',  pos_y='0.1mm', orientation='0'))
rt_meander = RouteMeander(design, 'readout',  Dict(
        total_length='4.18 mm',
        hfss_wire_bonds = False,
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='second_end'),
            #start_pin=Dict(component=stg.name, pin='short'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

In [5]:
#Setup the launchpad location and orientation.
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight


#launch_options1 = dict(pos_x='1.25mm', pos_y='0.9mm', orientation='180')

#lp = LaunchpadWirebond(design, 'P4_Q', options = launch_options1)

otg2 = OpenToGround(design, 'open_to_ground2', options=dict(pos_x='-0.9mm', pos_y='1.25mm',
orientation='90'))



#Setup the OpenToGround location and orientation.
#launch_options2 = dict(pos_x='-1.25mm',  pos_y='0.9mm', orientation='0')

#lp2 = LaunchpadWirebond(design, 'P5_Q', options = launch_options2)


otg3 = OpenToGround(design, 'open_to_ground3', options=dict(pos_x='-0.9mm', pos_y='-1.25mm',
orientation='270'))

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options_left = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_start'),
            end_pin=Dict(component=otg3.name, pin='open')), )

meander_options_right = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_end'),
            end_pin=Dict(component=otg2.name, pin='open')), )


meander_left = RouteStraight(design, 'cpw_left',options=meander_options_left)
meander_right = RouteStraight(design, 'cpw_right',options=meander_options_right)

gui.rebuild()
gui.autoscale()

In [6]:
from qiskit_metal.analyses.simulation import ScatteringImpedanceSim
em1 = ScatteringImpedanceSim(design, "hfss")

In [7]:
hfss = em1.renderer

In [8]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [9]:
hfss.connect_ansys()

INFO 05:32PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:32PM [load_ansys_project]: 	Opened Ansys App
INFO 05:32PM [load_ansys_project]: 	Opened Ansys Desktop v2019.1.0
INFO 05:32PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/2254729M/Documents/Ansoft/
	Project:   Project18
INFO 05:32PM [connect_design]: No active design found (or error getting active design).
INFO 05:32PM [connect]: 	 Connected to project "Project18". No design detected


In [10]:
hfss.activate_ansys_design("HangingResonators", 'drivenmodal')

05:32PM 50s WARNING [activate_ansys_design]: The design_name=HangingResonators was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 05:32PM [connect_design]: 	Opened active design
	Design:    HangingResonators [Solution type: DrivenModal]
WARNING 05:32PM [connect_setup]: 	No design setup detected.
WARNING 05:32PM [connect_setup]: 	Creating drivenmodal default setup.
INFO 05:32PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)


In [11]:
design._chips['main']['size']['size_y'] = '2.512mm'
design._chips['main']['size']['size_x'] = '2.75mm'

In [ ]:
hfss.render_design(['readout', 'open_to_ground', 'TQ1', otg2.name, otg3.name, 'cpw_left', 'cpw_right'], [],box_plus_buffer=False)

In [ ]:
hfss.clean_active_design()

In [12]:
hfss.render_design(selection=[],
                   open_pins=[],
                   port_list=[('cpw_right', 'end', 50), ('cpw_left', 'end', 50)],
                   jj_to_port=[],
                   ignored_jjs=[],
                   box_plus_buffer = False)

In [ ]:
hfss.add_sweep(setup_name="Setup",
               name="Sweep",
               start_ghz=4.0,
               stop_ghz=9.0,
               count=10001,
               type="Discrete")

In [ ]:
hfss.analyze_sweep('Sweep', 'Setup')

In [ ]:
hfss.plot_params(['S11', 'S21'])

In [13]:
hfss.disconnect_ansys()